In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os
import re
from matplotlib import pyplot as plt
import scipy.io as sio
import numpy as np
import pickle as pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from scipy.signal import spectrogram
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from scipy import signal
import scipy.io
import os
import math

import tensorflow as tf
import keras
from tensorflow.keras.utils import to_categorical
# from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, LSTM, Conv2D, MaxPooling2D, RepeatVector
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dataset_path = '/content/drive/My Drive/Major/BED_dataset_RAW_PARSED/'

In [ ]:
def load_eeg_data(directory,):
    data = []
    labels = []

    for filename in os.listdir(directory):
        if filename.endswith('.mat'):
            filepath = os.path.join(directory, filename)

            mat_contents = scipy.io.loadmat(filepath)
            recordings = mat_contents['recording']

            recordings = recordings[:30208, 10]
            recordings = recordings.T

            data.append(recordings)

            # Extract label from filename
            start_index = filename.index('s') + 1
            end_index = filename.index('_')
            # Extract the substring between 's' and '_'
            number = filename[start_index:end_index]
            person_id = int(number) # Convert the extracted number to an integer

            label = person_id - 1
            labels.append(label)

            # Augmentation - Gussian Noise
            noisy_data = add_gaussian_noise(recordings);
            data.append(noisy_data)
            labels.append(label)

    data = np.array(data)
    labels = np.array(labels)

    return data, labels

In [ ]:
eeg_data, labels = load_eeg_data(dataset_path)

In [ ]:
eeg_Label = []
for i in labels:
    for j in range (0,1):
        eeg_Label.append(i)

In [ ]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(eeg_Label)

In [ ]:
eeg_Data = eeg_data

In [ ]:
fs = 256

size_dataset = len(eeg_Data)
nperseg = 512
noverlap = 256
f_size = math.ceil(((nperseg + 1)/2))
t_size = int((eeg_Data[0].size - noverlap ) / (nperseg - noverlap))
X_full = np.ndarray(shape=(size_dataset, f_size, t_size, 3))

for i in range(0, size_dataset):
  for j in range(0,3):
    X = eeg_Data[i]
    f, t, Sxx = spectrogram(X, fs, nperseg = nperseg, noverlap = noverlap, mode='psd')
    X_full[i, :, :, j] = Sxx

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_full, y, test_size=0.2, random_state=42)

X_train /= 255
X_test /= 255

In [ ]:
num_classes = 21

input_shape = (X_train.shape[1],X_train.shape[2],X_train.shape[3])
input_shape

(257, 99, 3)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Calculate class weights based on imbalance ratio
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(labels), y=labels)

# Convert class_weights array to a dictionary
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3, 3),
          strides=(1, 1),
          activation='relu',
          input_shape=input_shape))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model.add(Conv2D(64, (3, 3),
          activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())

model.add(RepeatVector(4))

model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128))
model.add(Dropout(0.2))

model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))


model.add(Dense(num_classes, activation="softmax"))


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 255, 97, 32)       896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 48, 32)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 46, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 23, 64)        0         
 g2D)                                                            
                                                                 
 dropout (Dropout)           (None, 62, 23, 64)        0         
                                                                 
 flatten (Flatten)           (None, 91264)             0

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
mc = tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
model.compile(optimizer ="Adam", loss =keras.losses.categorical_crossentropy, metrics=['accuracy'])
m=model.fit(X_train, y_train,epochs=100,batch_size=256,verbose=1,validation_data=(X_test, y_test), class_weight=class_weights_dict, callbacks =[es, mc])

In [ ]:
saved_model = load_model('best_model.h5')

In [ ]:
_, train_f1 = saved_model.evaluate(X_train, y_train, verbose=0)
_, test_f1 = saved_model.evaluate(X_test, y_test, verbose=0)

print(test_f1)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(m.history['accuracy'])
plt.plot(m.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='best')
plt.show()

In [ ]:
y_test1=np.argmax(y_test, axis=1)

y_pred_saved = saved_model.predict(X_test)
y_pred_saved=np.argmax(y_pred_saved,axis=1)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test1,y_pred_saved))

In [ ]:
saved_model = load_model('best_model.h5')
saved_model.save('/content/drive/My Drive/saved_models/BED_P4.h5')